In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Model, Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from keras import layers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

dataset_path = '/content/drive/MyDrive/PokemonData'

# Define image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32

# Create data generators for training, validation, and testing
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Set the validation split
)

train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

test_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 5511 images belonging to 150 classes.
Found 1309 images belonging to 150 classes.
Found 1309 images belonging to 150 classes.


In [ ]:
 # Create a CNN model
model = Sequential([
    Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(512, activation='relu', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(150, activation='softmax')
])
# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Set up early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[early_stopping, reduce_lr]
)

test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f'Test accuracy: {test_acc}')

model.save('pokemon_classifier.h5')

Epoch 1/100


KeyboardInterrupt: 

In [ ]:
print(f'Test accuracy: {test_acc}')

Test accuracy: 0.7484375238418579


Attempt #2 with using a VGG16 model

In [ ]:
from tensorflow.keras.applications import VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

for layer in base_model.layers:
    layer.trainable = False

model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(150, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Set up early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)

model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[early_stopping, reduce_lr]
)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f'Test accuracy: {test_acc}')

# Save the model
model.save('pokemon_classifier2.0.h5')

Epoch 1/100
172/172 [==============================] - 72s 411ms/step - loss: 7.2342 - accuracy: 0.3716 - val_loss: 4.3731 - val_accuracy: 0.5891 - lr: 0.0010
Epoch 2/100
172/172 [==============================] - 71s 414ms/step - loss: 3.5114 - accuracy: 0.6704 - val_loss: 3.4300 - val_accuracy: 0.6023 - lr: 0.0010
Epoch 3/100
172/172 [==============================] - 69s 403ms/step - loss: 2.8839 - accuracy: 0.7176 - val_loss: 3.1480 - val_accuracy: 0.6602 - lr: 0.0010
Epoch 4/100
172/172 [==============================] - 71s 412ms/step - loss: 2.6976 - accuracy: 0.7281 - val_loss: 3.1232 - val_accuracy: 0.6078 - lr: 0.0010
Epoch 5/100
172/172 [==============================] - 70s 404ms/step - loss: 2.6298 - accuracy: 0.7324 - val_loss: 3.1043 - val_accuracy: 0.6195 - lr: 0.0010
Epoch 6/100
172/172 [==============================] - 69s 403ms/step - loss: 2.6412 - accuracy: 0.7260 - val_loss: 3.2134 - val_accuracy: 0.5844 - lr: 0.0010
Epoch 7/100
172/172 [=========================

Attempt #3 with changed parameters

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import VGG16
from tensorflow.keras.regularizers import l2

dataset_path = '/content/drive/MyDrive/PokemonData'

# Define image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32

# Create data generators for training, validation, and testing with aggressive data augmentation
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    dataset_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Ensure data isn't shuffled for evaluation
)
# Load VGG16 base model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Fine-tune the last few layers of the base model
for layer in base_model.layers[:-4]:
    layer.trainable = False

# Build the model with additional layers and regularization
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(512, activation='relu', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(150, activation='softmax')
])

# Compile the model with a lower learning rate
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Set up early stopping and learning rate reduction callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1)

# Train the model with increased epochs
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=1000,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[early_stopping, reduce_lr]
)

# Evaluate the model on the test set
try:
    test_loss, test_acc = model.evaluate(test_generator)
    print(f'Test accuracy: {test_acc}')
except ValueError as e:
    print("Error evaluating the model on the test set:", e)

# Save the model
model.save('pokemon_classifier3.0_improved.h5')


Found 5511 images belonging to 150 classes.
Found 1309 images belonging to 150 classes.
Found 6820 images belonging to 150 classes.
58889256/58889256 [==============================] - 4s 0us/step


Epoch 1/1000
172/172 [==============================] - 4770s 28s/step - loss: 12.6891 - accuracy: 0.0113 - val_loss: 8.7344 - val_accuracy: 0.0227 - lr: 0.0010
Epoch 2/1000
172/172 [==============================] - 66s 384ms/step - loss: 8.1612 - accuracy: 0.0124 - val_loss: 8.2573 - val_accuracy: 0.0086 - lr: 0.0010
Epoch 3/1000
172/172 [==============================] - 66s 383ms/step - loss: 6.7885 - accuracy: 0.0173 - val_loss: 8.9754 - val_accuracy: 0.0055 - lr: 0.0010
Epoch 4/1000
172/172 [==============================] - 65s 381ms/step - loss: 6.1043 - accuracy: 0.0214 - val_loss: 7.5873 - val_accuracy: 0.0102 - lr: 0.0010
Epoch 5/1000
172/172 [==============================] - 65s 380ms/step - loss: 5.8897 - accuracy: 0.0184 - val_loss: 6.4643 - val_accuracy: 0.0305 - lr: 0.0010
Epoch 6/1000
172/172 [==============================] - 66s 383ms/step - loss: 5.7507 - accuracy: 0.0257 - val_loss: 9538.3213 - val_accuracy: 0.0078 - lr: 0.0010
Epoch 7/1000
172/172 [==============

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Attempt #4 changing parameters again!

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import VGG16
from tensorflow.keras.regularizers import l2

dataset_path = '/content/drive/MyDrive/PokemonData'

# Define image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32

# Create data generators for training, validation, and testing with aggressive data augmentation
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    dataset_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Ensure data isn't shuffled for evaluation
)
# Load VGG16 base model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Fine-tune the last few layers of the base model
for layer in base_model.layers[:-4]:
    layer.trainable = False

# Build the model with additional layers and regularization
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),  # Removed kernel_regularizer=l2(0.01)
    BatchNormalization(),
    Dropout(0.4),  # Reduced dropout rate
    Dense(512, activation='relu'),  # Removed kernel_regularizer=l2(0.01)
    BatchNormalization(),
    Dropout(0.4),  # Reduced dropout rate
    Dense(150, activation='softmax')
])

# Increasing learning rate
model.compile(optimizer=Adam(lr=0.0005), loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tuning more VGG16 layers
for layer in base_model.layers[:-10]:  # Fine-tune more layers
    layer.trainable = True

# Increased patience for early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Increased factor for reducing learning rate
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, verbose=1)

# Train the model with increased epochs
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=1000,  # Increased epochs
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[early_stopping, reduce_lr]
)

# Evaluate the model on the test set
try:
    test_loss, test_acc = model.evaluate(test_generator)
    print(f'Test accuracy: {test_acc}')
except ValueError as e:
    print("Error evaluating the model on the test set:", e)

# Save the model
model.save('pokemon_classifier3.0_improved.h5')


Found 5511 images belonging to 150 classes.
Found 1309 images belonging to 150 classes.
Found 6820 images belonging to 150 classes.


Epoch 1/1000
172/172 [==============================] - 67s 375ms/step - loss: 5.5907 - accuracy: 0.0151 - val_loss: 5.1893 - val_accuracy: 0.0164 - lr: 0.0010
Epoch 2/1000
172/172 [==============================] - 65s 374ms/step - loss: 4.9913 - accuracy: 0.0316 - val_loss: 4.8733 - val_accuracy: 0.0437 - lr: 0.0010
Epoch 3/1000
172/172 [==============================] - 66s 382ms/step - loss: 4.6466 - accuracy: 0.0489 - val_loss: 13.2918 - val_accuracy: 0.0125 - lr: 0.0010
Epoch 4/1000
172/172 [==============================] - 66s 381ms/step - loss: 4.2176 - accuracy: 0.0730 - val_loss: 5.9835 - val_accuracy: 0.0398 - lr: 0.0010
Epoch 5/1000
172/172 [==============================] - 64s 373ms/step - loss: 3.9424 - accuracy: 0.0976 - val_loss: 5.2378 - val_accuracy: 0.0430 - lr: 0.0010
Epoch 6/1000
172/172 [==============================] - 64s 372ms/step - loss: 3.6822 - accuracy: 0.1267 - val_loss: 4.9565 - val_accuracy: 0.0719 - lr: 0.0010
Epoch 7/1000
172/172 [=================

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os

model_path = 'pokemon_classifier3.0_improved.h5'
model = load_model(model_path)

# Define image dimensions
img_width, img_height = 150, 150

class_labels_file = '/content/drive/MyDrive/PokemonData/class_labels.txt'

# Read class labels from class_labels.txt file
with open(class_labels_file, 'r') as f:
    class_labels = f.readlines()
    # Remove newline characters from the end of each label
    class_labels = [label.strip() for label in class_labels]

def preprocess_image(image_path):
    # Load and preprocess the image
    img = cv2.imread(image_path)
    img = cv2.resize(img, (img_width, img_height))
    img = img / 255.0  # Normalize pixel values
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

def predict_image(image_path):
    # Preprocess the image
    img = preprocess_image(image_path)
    # Make predictions
    predictions = model.predict(img)
    # Get the predicted class label
    predicted_label = class_labels[np.argmax(predictions)]
    # Get the confidence score
    confidence = np.max(predictions) * 100
    return predicted_label, confidence

# Test your model with images
image_paths = ['/content/006.png']

for image_path in image_paths:
    predicted_label, confidence = predict_image(image_path)
    print(f"Predicted label: {predicted_label}, Confidence: {confidence:.2f}%")


1/1 [==============================] - 0s 176ms/step
Predicted label: Blastoise, Confidence: 93.50%
